In [ ]:
! pip install pandas numpy faker

In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [7]:
def generate_bank_transactions(num_users, start_date, end_date, output_file="bank_transactions.csv"):
    
    # 사용자 번호 리스트 생성
    users_id = [f"user_{i+1}" for i in range(num_users)]
    
    # 사용자별 고정 금액 할당
    user_salaries = {user_id: np.random.randint(2500000, 5000000) for user_id in users_id}
    loan_repayment = {user_id: np.random.randint(0, 500000) for user_id in users_id}
    
    # 상시 입출금 항목 리스트 및 금액 범위
    transaction_types = {
        '입금:보험금': (10000, 200000),
        '입금:배당금': (1000, 100000),
        '출금:용돈': (-500000, -50000)
    }
    

    # 데이터프레임 초기화
    transaction_data = {
        'User_ID': [],
        'Date': [],
        'Transaction_Type': [],
        'Amount': []
    }
    
    # 입출금 내역 생성
    for user_id in users_id:
        current_date = start_date

        while current_date <= end_date:

            # 매월 10일 월급 고정입금
            if current_date.day == 10:
                transaction_data['User_ID'].append(user_id)
                transaction_data['Date'].append(current_date)
                transaction_data['Transaction_Type'].append('입금:월급')
                transaction_data['Amount'].append(user_salaries[user_id])  # 미리 할당된 월급 금액
            
            # 매월 1일 대출금 고정출금
            if current_date.day == 1:
                transaction_data['User_ID'].append(user_id)
                transaction_data['Date'].append(current_date)
                transaction_data['Transaction_Type'].append('출금:대출상환')
                transaction_data['Amount'].append(-loan_repayment[user_id])  # 미리 할당된 금액
                # transaction_data['Amount'].append(-np.random.randint(500000, 2000000))  # 매번랜덤
            
            # 해당 월의 랜덤 입출금 내역 생성
            if current_date.day == 1:
                num_transactions = np.random.randint(0, 2)  # 한 달에 0 ~ 2건의 거래 발생
                for _ in range(num_transactions):
                    lastday_of_month = (current_date.replace(month=current_date.month % 12 + 1, day=1) - timedelta(days=1)).day
                    transaction_day = random.randint(1, lastday_of_month)  # 해당 월의 마지막 날을 기준으로 랜덤 날짜 선택
                    transaction_date = current_date.replace(day=transaction_day)
                    transaction_type = random.choice(list(transaction_types.keys()))
                    amount_range = transaction_types[transaction_type]
                    amount = np.random.randint(amount_range[0], amount_range[1])
                    
                    transaction_data['User_ID'].append(user_id)
                    transaction_data['Date'].append(transaction_date)
                    transaction_data['Transaction_Type'].append(transaction_type)
                    transaction_data['Amount'].append(amount)

            current_date += timedelta(days=1)


    # 데이터프레임 생성
    transaction_df = pd.DataFrame(transaction_data)
    
    # CSV 파일로 저장
    transaction_df.to_csv(f'./data/{output_file}', index=False, encoding='cp949')
    
    return transaction_df

# 사용 예시
num_users = 1
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# 함수 호출
transaction_df = generate_bank_transactions(num_users, start_date, end_date)
print(transaction_df.head())  # 데이터프레임의 첫 몇 줄을 출력

  User_ID       Date Transaction_Type   Amount
0  user_1 2023-01-01          출금:대출상환   -71050
1  user_1 2023-01-24           입금:배당금    78855
2  user_1 2023-01-10            입금:월급  3675195
3  user_1 2023-02-01          출금:대출상환   -71050
4  user_1 2023-02-10            입금:월급  3675195
